# Notebook to create indicators modules

In [66]:
from pathlib import Path

ROOTDIR = Path.cwd().parent
PKGDIR = ROOTDIR.joinpath("src/mintalib").resolve(strict=True)


In [67]:
from mintalib.samples import sample_prices

prices = sample_prices()
prices


,open,high,low,close,volume
date,,,,,
1980-12-12,0.098943,0.099373,0.098943,0.098943,469033600
1980-12-15,0.094211,0.094211,0.093781,0.093781,175884800
1980-12-16,0.087328,0.087328,0.086898,0.086898,105728000
1980-12-17,0.089049,0.089479,0.089049,0.089049,86441600
1980-12-18,0.091630,0.092061,0.091630,0.091630,73449600
...,...,...,...,...,...
2024-10-15,233.610001,237.490005,232.369995,233.850006,64751400
2024-10-16,231.600006,232.119995,229.839996,231.779999,34082200
2024-10-17,233.429993,233.850006,230.520004,232.149994,32993800


In [68]:
"""
Factory functions for technical analysis indicators.

Indicator factory names are all upper case.

Indicators offer a composable interface where a calculation routine
is bound together with its calculation parameters.

An indicator object is a callable that can be applied to prices or series data.

Indicators can be chained with the `@` operator as in `ROC(1) @ SMA(20)`.

The `@` operator can also be used to apply an indicator to its parameter.

So for example `SMA(50) @ prices` can be used to compute the 50 period simple moving average on `prices`,
instead of the more verbose `SMA(50)(prices)`.
"""

'\nFactory functions for technical analysis indicators.\n\nIndicator factory names are all upper case.\n\nIndicators offer a composable interface where a calculation routine\nis bound together with its calculation parameters.\n\nAn indicator object is a callable that can be applied to prices or series data.\n\nIndicators can be chained with the `@` operator as in `ROC(1) @ SMA(20)`.\n\nThe `@` operator can also be used to apply an indicator to its parameter.\n\nSo for example `SMA(50) @ prices` can be used to compute the 50 period simple moving average on `prices`,\ninstead of the more verbose `SMA(50)(prices)`.\n'

In [ ]:
# PREAMBLE Do not edit! This file was generated

import inspect

from mintalib import core
from mintalib.model import FuncIndicator

nan = float('nan')

def wrap_indicator(calc_func):
    """decorator to update indicator with documentation from source"""

    def decorator(func):
        name = func.__name__
        sig = inspect.signature(func)

        def wrapper(*args, **kwargs):
            binding = sig.bind(*args, **kwargs)
            binding.apply_defaults()
            params = dict(binding.arguments)

            return FuncIndicator(
                name=name,
                func=calc_func,
                params=params,
            )

        wrapper.__name__ = func.__name__
        wrapper.__doc__ = calc_func.__doc__
        wrapper.__signature__ = sig

        return wrapper

    return decorator


In [70]:
@wrap_indicator(core.calc_sma)
def SMA(period: int, *, item: str = None): ...

SMA(5) @ prices

date
1980-12-12           NaN
1980-12-15           NaN
1980-12-16           NaN
1980-12-17           NaN
1980-12-18      0.092060
                 ...    
2024-10-15    230.256000
2024-10-16    230.704001
2024-10-17    231.326001
2024-10-18    232.816000
2024-10-21    233.851999
Length: 11056, dtype: float64

In [71]:
import inspect

from mintalib.builder import annotate_parameter

def make_signature(calc_func):
    """creates function signature from core function"""
    
    sig = inspect.signature(calc_func)
    first_param = next(iter(sig.parameters.values()))
    
    new_params = []
    for param in sig.parameters.values():
        if param.name in ("series", "prices", "wrap"):
            continue
        param = annotate_parameter(param)
        new_params.append(param)

    if first_param.name == "series":
        item_param = inspect.Parameter(
            name="item",
            kind=inspect.Parameter.KEYWORD_ONLY,
            default=None,
            annotation=str
        )
        new_params.append(item_param)

    return sig.replace(parameters=new_params)

sig = make_signature(core.calc_sma)
print(sig)


(period: int, *, item: str = None)


In [72]:
def make_indicator(calc_func, name=None):
    if name is None:
        name = calc_func.__name__.removeprefix("calc_").upper()
    cname = f"core.{calc_func.__name__}"
    newsig = make_signature(calc_func)
    buffer = f"@wrap_indicator({cname})\n"
    buffer += f"def {name}{newsig}: ...\n"
    return buffer

output = make_indicator(core.calc_sma)
print(output)

@wrap_indicator(core.calc_sma)
def SMA(period: int, *, item: str = None): ...



In [73]:
def core_functions():
    return sorted(k for k, v in vars(core).items() if k.startswith("calc") and callable(v))

core_functions()

['calc_abs',
 'calc_adx',
 'calc_alma',
 'calc_atr',
 'calc_avgprice',
 'calc_bbands',
 'calc_bop',
 'calc_cci',
 'calc_clag',
 'calc_cmf',
 'calc_crossover',
 'calc_crossunder',
 'calc_curve',
 'calc_dema',
 'calc_diff',
 'calc_dmi',
 'calc_ema',
 'calc_eval',
 'calc_exp',
 'calc_flag',
 'calc_hma',
 'calc_kama',
 'calc_keltner',
 'calc_ker',
 'calc_lag',
 'calc_log',
 'calc_lroc',
 'calc_macd',
 'calc_mad',
 'calc_mav',
 'calc_max',
 'calc_mdi',
 'calc_mfi',
 'calc_midprice',
 'calc_min',
 'calc_natr',
 'calc_pdi',
 'calc_ppo',
 'calc_price',
 'calc_qsf',
 'calc_rma',
 'calc_roc',
 'calc_rsi',
 'calc_rvalue',
 'calc_sar',
 'calc_shift',
 'calc_sign',
 'calc_slope',
 'calc_sma',
 'calc_stdev',
 'calc_step',
 'calc_stoch',
 'calc_streak',
 'calc_sum',
 'calc_tema',
 'calc_trange',
 'calc_tsf',
 'calc_typprice',
 'calc_updown',
 'calc_wclprice',
 'calc_wma']

In [74]:
import re
from pprint import pformat

import importlib.util

def get_history(pattern):
    cells = [c for c in _ih if re.match(pattern, c)]
    for cell in cells:
        pass
    return cell


HEREDOCS = get_history('"""')
PREAMBLE = get_history('# PREAMBLE')
PREAMBLE = re.sub(r'^from mintalib\.?', 'from .', PREAMBLE, flags=re.MULTILINE)

def make_indicators(cnames=None):
    if cnames is None:
        cnames = core_functions()

    output = ""

    if HEREDOCS:
        output += HEREDOCS + "\n\n"

    if PREAMBLE:
        output += PREAMBLE + "\n\n"

    fnames = []

    for cname in cnames:
        cfunc = getattr(core, cname)
        name = cname.removeprefix("calc_").upper()
        code = make_indicator(cfunc, name)
        fnames.append(name)
        output += code + "\n"

    # output += "__all__ = [name for name in dir() if name.isupper()]\n"
    xnames = pformat(fnames, width=75, compact=True, indent=4)
    xnames = xnames.replace("[", " ").replace("]", "")
    output += f"__all__ = [\n{xnames}\n]\n"

    return output

output = make_indicators()
print(output)

def new_module(name: str, code: str = None):
    spec = importlib.util.spec_from_loader(name, None)
    module = importlib.util.module_from_spec(spec)
    if code:
        exec(code, module.__dict__)
    return module


indicators = new_module("mintalib.indicators", output)
indicators.__all__

"""
Factory functions for technical analysis indicators.

Indicator factory names are all upper case.

Indicators offer a composable interface where a calculation routine
is bound together with its calculation parameters.

An indicator object is a callable that can be applied to prices or series data.

Indicators can be chained with the `@` operator as in `ROC(1) @ SMA(20)`.

The `@` operator can also be used to apply an indicator to its parameter.

So for example `SMA(50) @ prices` can be used to compute the 50 period simple moving average on `prices`,
instead of the more verbose `SMA(50)(prices)`.
"""

# PREAMBLE Do not edit! This file was generated

import inspect

from . import core
from .core import wrap_indicator
from .model import FuncIndicator

nan = float('nan')

def wrap_indicator(calc_func):
    """decorator to update indicator with documentation from source"""

    def decorator(func):
        name = func.__name__
        sig = inspect.signature(func)

        def wrapper(*

['ABS',
 'ADX',
 'ALMA',
 'ATR',
 'AVGPRICE',
 'BBANDS',
 'BOP',
 'CCI',
 'CLAG',
 'CMF',
 'CROSSOVER',
 'CROSSUNDER',
 'CURVE',
 'DEMA',
 'DIFF',
 'DMI',
 'EMA',
 'EVAL',
 'EXP',
 'FLAG',
 'HMA',
 'KAMA',
 'KELTNER',
 'KER',
 'LAG',
 'LOG',
 'LROC',
 'MACD',
 'MAD',
 'MAV',
 'MAX',
 'MDI',
 'MFI',
 'MIDPRICE',
 'MIN',
 'NATR',
 'PDI',
 'PPO',
 'PRICE',
 'QSF',
 'RMA',
 'ROC',
 'RSI',
 'RVALUE',
 'SAR',
 'SHIFT',
 'SIGN',
 'SLOPE',
 'SMA',
 'STDEV',
 'STEP',
 'STOCH',
 'STREAK',
 'SUM',
 'TEMA',
 'TRANGE',
 'TSF',
 'TYPPRICE',
 'UPDOWN',
 'WCLPRICE',
 'WMA']

In [75]:
indicators.SMA(period=5, item='close') @ prices


date
1980-12-12           NaN
1980-12-15           NaN
1980-12-16           NaN
1980-12-17           NaN
1980-12-18      0.092060
                 ...    
2024-10-15    230.256000
2024-10-16    230.704001
2024-10-17    231.326001
2024-10-18    232.816000
2024-10-21    233.851999
Length: 11056, dtype: float64

In [76]:
indicators = new_module("mintalib.indicators", output)

outfile = PKGDIR / "indicators.py"

print(f"Updating {outfile.name} ...")

outfile.write_text(output)

Updating indicators.py ...


7019

In [19]:
core.calc_ema.__signature__

AttributeError: '_cython_3_1_2amsendbackport.cython_function_or_met' object has no attribute '__signature__'

In [20]:
def hello(name: str = "World") -> str:
    return f"Hello, {name}!"

hello.__signature__

AttributeError: 'function' object has no attribute '__signature__'